In [0]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.callbacks import TensorBoard
import numpy as np
import os
from keras import optimizers
import math
import matplotlib.pyplot as plt
from keras import losses


In [0]:
# Training parameters
num_classes = 10
batch_size = 128  
epochs = 10

activation = 'relu'
subtract_pixel_mean = True #Normalization
batch_normalization_status = True

loss='categorical_crossentropy'

learning_rate =5e-2
lr_scheduler_strategy = True

conv_first=True

metrics = ['accuracy']
data_augmentation = True


In [0]:
if lr_scheduler_strategy:
  def lr_schedule(epoch):
#     initial_lrate = 0.05
#     k = 0.1
#     lrate = initial_lrate *( np.exp(-k*epoch))
#     return lrate
    initial_lrate = 0.05
    drop = 0.25
    epochs_drop = 8
    lrate = initial_lrate * math.pow(drop,math.floor((1+epoch)/epochs_drop))
    print('Learning rate: ', lrate) 
    return lrate

else:
   def lr_schedule(epoch):
      lr=learning_rate
      print('Learning rate: ', lr)
      return lr
    
#   lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                  cooldown=0,
#                                  patience=5,
#                                  min_lr=0.5e-6)    
#lrate = initial_lrate *( np.exp(-k*epoch))
#     return lrate
score = (0.690527972300293, 0.8730)
#   lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                  cooldown=0,
#                                  patience=5,
#                                  min_lr=0.5e-6)

if lr_scheduler_strategy:
  sgd = optimizers.SGD(lr=0.0, momentum=0.9, decay=0.0, nesterov=False)
  optimizer=sgd

  lr_scheduler = LearningRateScheduler(lr_schedule)

#   lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
#                                  cooldown=0,
#                                  patience=5,
#                                  min_lr=0.5e-6)

  callbacks = [lr_scheduler]
  
else:
  optimizer='adadelta'
  lr_scheduler = LearningRateScheduler(lr_schedule)
  callbacks = [lr_scheduler]


In [0]:
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation=activation,
                 batch_normalization=batch_normalization_status,
                 conv_first=conv_first):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


In [0]:
def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [29]:
model = resnet_v1(input_shape=(32,32,3), depth=20)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_43 (Conv2D)              (None, 32, 32, 16)   448         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_39 (BatchNo (None, 32, 32, 16)   64          conv2d_43[0][0]                  
__________________________________________________________________________________________________
activation_39 (Activation)      (None, 32, 32, 16)   0           batch_normalization_39[0][0]     
__________________________________________________________________________________________________
conv2d_44 

In [0]:
from keras.datasets import cifar10

In [0]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [32]:
if subtract_pixel_mean:
    print("siddu")
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

siddu


In [0]:

model.compile(loss=loss,
                optimizer=optimizer,
                metrics=metrics)


In [34]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format='channels_first',
        validation_split=0.0)

    datagen.fit(x_train)

    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs,samples_per_epoch=len(x_train), verbose=1, workers=4,
                        callbacks=callbacks)

Using real-time data augmentation.


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:923: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_first" (channels on axis 1), i.e. expected either 1, 3 or 4 channels on axis 1. However, it was passed an array with shape (50000, 32, 32, 3) (32 channels).
  ' channels).')
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/numpy_array_iterator.py:127: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_first" (channels on axis 1), i.e. expected either 1, 3, or 4 channels on axis 1. However, it was passed an array with shape (50000, 32, 32, 3) (32 channels).
  str(self.x.shape[channels_axis]) + ' channels).')
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=(array([[[..., epochs=10, verbose=1, workers=4, callbacks=[<ker

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
Learning rate:  0.05
390/390 [==============================] - 92s 236ms/step - loss: 1.7661 - acc: 0.4151 - val_loss: 1.5289 - val_acc: 0.5185
Epoch 2/10
Learning rate:  0.05
390/390 [==============================] - 85s 217ms/step - loss: 1.2533 - acc: 0.6111 - val_loss: 1.2774 - val_acc: 0.6059
Epoch 3/10
Learning rate:  0.05
390/390 [==============================] - 87s 224ms/step - loss: 1.0500 - acc: 0.6835 - val_loss: 1.3903 - val_acc: 0.6010
Epoch 4/10
Learning rate:  0.05
390/390 [==============================] - 85s 218ms/step - loss: 0.9238 - acc: 0.7329 - val_loss: 1.1948 - val_acc: 0.6646
Epoch 5/10
Learning rate:  0.05
390/390 [==============================] - 86s 219ms/step - loss: 0.8337 - acc: 0.7656 - val_loss: 0.9916 - val_acc: 0.7273
Epoch 6/10
Learning rate:  0.05
390/390 [==============================] - 87s 224ms/step - loss: 0.7724 - acc: 0.7873 - val_loss: 0.9339 - val_acc: 0.7297
Epoch 7/10
Learn

In [7]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.630527995300293
Test accuracy: 0.8762


In [37]:
print(history.history.keys()) 
plt.plot(history1.history['acc']) 
plt.plot(history1.history['val_acc']) 


plt.legend(['acc', 'val_acc'], loc='upper left')

NameError: ignored